In [197]:

import os
import copy
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM, BartForConditionalGeneration,BartTokenizer
import shap
import scipy as sp
import nlp
import torch
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option("max_rows", None)
from shap.utils import assert_import, record_import_error, safe_isinstance, make_masks, OpChain, MaskedModel
from shap import maskers, links

In [139]:
tokenizer2 = BartTokenizer.from_pretrained("sshleifer/distilbart-xsum-12-6")
model2 =  BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-xsum-12-6")

In [177]:
tokenizer3 = BartTokenizer.from_pretrained("sshleifer/distilbart-xsum-12-1")
model3 =  BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-xsum-12-1")

In [3]:
dataset = nlp.load_dataset('xsum',split='train')

WARNING - Using custom data configuration default


In [179]:
def summarize(ARTICLE_TO_SUMMARIZE,model,tokenizer):
    print(f"Len of article: {len(ARTICLE_TO_SUMMARIZE.split(' '))}")
    inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')
    summary_ids = model.generate(inputs['input_ids'])
    summary=[tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    print(summary)
    print(f"Len of summary: {len(summary[0].split(' '))}")

In [195]:
ARTICLE_TO_SUMMARIZE=dataset['document'][0]
summarize(ARTICLE_TO_SUMMARIZE,model3,tokenizer3)

WARNING - Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Len of article: 176
[' Sony has said it is working to fix a problem with the PlayStation Network.']
Len of summary: 15


In [196]:
ARTICLE_TO_SUMMARIZE=dataset['document'][0]
summarize(ARTICLE_TO_SUMMARIZE,model2,tokenizer2)

WARNING - Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Len of article: 176
['Sony has said that a bug in its PlayStation 3 console is causing problems with the PlayStation Network.']
Len of summary: 18


In [194]:
dataset['document'][0]

'The problem is affecting people using the older versions of the PlayStation 3, called the "Fat" model.The problem isn\'t affecting the newer PS3 Slim systems that have been on sale since September last year.Sony have also said they are aiming to have the problem fixed shortly but is advising some users to avoid using their console for the time being."We hope to resolve this problem within the next 24 hours," a statement reads. "In the meantime, if you have a model other than the new slim PS3, we advise that you do not use your PS3 system, as doing so may result in errors in some functionality, such as recording obtained trophies, and not being able to restore certain data."We believe we have identified that this problem is being caused by a bug in the clock functionality incorporated in the system."The PlayStation Network is used by millions of people around the world.It allows users to play their friends at games like Fifa over the internet and also do things like download software o

In [190]:
dataset['summary'][14]

'The winners of a TV talent show have beaten the Pope to the album of the year at the Classical Brit Awards.\n'

In [198]:
fm = MaskedModel(model, masker, link)

NameError: name 'self' is not defined